In [18]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate,ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser,PydanticOutputParser
from pydantic import BaseModel,Field
from typing import List
from IPython.display import display,display_markdown,display_json
from pydantic import SecretStr
from dotenv import load_dotenv
# 加载 .env 文件
load_dotenv()

True

In [11]:
model=ChatOpenAI(model="qwen-turbo",
                 api_key=SecretStr(os.getenv("DASHSCOPE_API_KEY")),
                 base_url=os.getenv("BASE_URL"),
                 temperature=0.3,
                 frequency_penalty=1.5
                )

In [12]:
class BookInfo(BaseModel):
    book_name: str=Field(
        description="书籍的名字",
        examples="百年孤独"
    )
    author_name:str=Field(
        description="书籍的作者",
        examples="加西亚-马克克斯"
          )
    generes:List[str]=Field(
        description="书籍的体裁",
        examples=["小说","文学"]
    )

In [23]:
output_parser=PydanticOutputParser(pydantic_object=BookInfo)
parser_instructions=output_parser.get_format_instructions()
display(output_parser.get_format_instructions())

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"book_name": {"description": "书籍的名字", "examples": "百年孤独", "title": "Book Name", "type": "string"}, "author_name": {"description": "书籍的作者", "examples": "加西亚-马克克斯", "title": "Author Name", "type": "string"}, "generes": {"description": "书籍的体裁", "examples": ["小说", "文学"], "items": {"type": "string"}, "title": "Generes", "type": "array"}}, "required": ["book_name", "author_name", "generes"]}\n```'

In [14]:
prompt=ChatPromptTemplate.from_messages([
    ("system","{parser_instructions} 你输出的结果请使用中文"),
    ("human","请你帮我从书籍概述中,提取书名,作者,以及书籍的体裁.书籍概述会被三个#符号包围\n###{book_introduction}###")
])

In [15]:
book_introduction=""" 
《明朝那些事儿》百科介绍
基本信息
书名：明朝那些事儿
作者：当年明月（本名石悦）
出版时间：2006年首次出版
类别：历史小说、通俗历史读物
篇幅：共九部，总计数百万字
内容简介
《明朝那些事儿》是一部以明朝历史为背景的长篇历史小说。它从1344年开始，一直讲述到1644年明朝灭亡，涵盖了将近三百年的时间跨度。书中以明朝十六位皇帝为核心，同时描绘了众多王公权贵、文臣武将以及普通小人物的命运，展现了明朝社会的复杂面貌。

该书以严谨的史料为基础，结合生动的小说笔法，用幽默风趣的语言讲述了明朝的政治、军事、经济和文化等方面的历史事件。尤其对官场斗争、战争场面、帝王心术等内容着墨较多，使读者能够更加直观地理解历史人物的心理活动与时代背景。

创作特点
语言风格：轻松幽默，打破传统历史书籍的枯燥感，让历史变得有趣。
叙事方式：以年代为主线，结合具体人物故事，穿插历史事件，形成全景式的历史画卷。
史料基础：作者通过大量查阅正史资料（如《明史》《二十四史》等），确保内容在趣味性之外兼具一定的历史准确性。
受众广泛：适合对历史感兴趣的大众读者，尤其是年轻群体，作为了解明朝历史的入门读物。
社会影响
《明朝那些事儿》自问世以来，广受读者欢迎，被誉为中国现代通俗历史文学的经典之作。
它不仅在国内畅销多年，还被翻译成多种语言，在海外也拥有大量读者。
该书的成功推动了“草根历史”写作潮流，激发了许多人对历史学习的兴趣。
作者简介
笔名：当年明月
原名：石悦
职业：作家、公务员
代表作品：《明朝那些事儿》是其最著名的作品，也是他以业余时间创作的成果。
相关评价
正面评价：许多读者称赞该书将枯燥的历史变得生动有趣，易于理解，是一本“让人笑着学历史”的好书。
批评观点：也有部分学者认为书中部分内容存在过度演绎或主观色彩较强的问题，建议将其视为“历史小说”而非完全意义上的学术著作。
出版形式
纸质书籍：分为多卷出版，涵盖整个明朝历史。
电子书与有声书：由于市场需求大，该书也被制作成电子书及有声读物，方便不同人群阅读/收听。
改编作品：已被改编为电视剧、广播剧等多种形式。
总结
《明朝那些事儿》以其独特的写作风格和深入浅出的历史解读方式，成为当代中国最受欢迎的历史类畅销书之一。它不仅让广大读者爱上历史，也为历史普及工作做出了重要贡献。
"""

In [24]:
final_prompt=prompt.invoke({
    "book_introduction":book_introduction,
    "parser_instructions":parser_instructions
})

In [25]:
response=model.invoke(final_prompt)
display_json(response.content,raw=True)

In [27]:
result=output_parser.invoke(response)
display(result)

BookInfo(book_name='明朝那些事儿', author_name='当年明月', generes=['历史小说', '通俗历史读物'])